In [3]:
import pandas as pd
import numpy as np
import os,os.path
from joblib import Parallel, delayed
import warnings
from dateutil.relativedelta import relativedelta
import collections.abc
import datetime

In [1]:
data_path = "./20221206/000002.XSHE.parquet"

In [4]:
aa = pd.read_parquet(f'{data_path}')

In [6]:
aa.head()

,securityid,updatetime,tradp,tradv,buyno,sellno,bs,localtime,date
34049700,000002.XSHE,09:15:00.030,0.0,100.0,2734,0,B,09:15:00.069,20221206
34049746,000002.XSHE,09:15:00.060,0.0,100.0,0,974,S,09:15:00.210,20221206
34050106,000002.XSHE,09:15:00.620,0.0,1000.0,6460,0,B,09:15:01.933,20221206
34050326,000002.XSHE,09:15:00.850,0.0,2000.0,0,85086,S,09:15:02.308,20221206
34050377,000002.XSHE,09:15:00.880,0.0,100.0,87618,0,B,09:15:02.370,20221206


In [28]:
aa["tmp"] = aa['date'].apply(str)+' '+aa['updatetime'].apply(str)

In [29]:
aa["tmp"].head()

34109606    20221206 09:25:00.000
34109607    20221206 09:25:00.000
34109608    20221206 09:25:00.000
34109609    20221206 09:25:00.000
34109610    20221206 09:25:00.000
Name: tmp, dtype: object

In [9]:
date_t = pd.to_datetime(aa["tmp"])

In [12]:
x = pd.to_datetime(f"2022-12-06 09:15:00.030")

In [22]:
aa['datetime'] = pd.to_datetime(aa['date'].apply(str)+' '+aa['updatetime'].apply(str))
# select
aa = aa.loc[aa.tradp>0]
# 
aa['tradamt'] = aa['tradv'] * aa['tradp']
aa_B = aa.loc[aa.bs == 'B']
aa_S = aa.loc[aa.bs == 'S']

/tmp/ipykernel_63493/4013361079.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['tradamt'] = aa['tradv'] * aa['tradp']


In [30]:
test = aa.head(10)

In [38]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index = ['d', 'a', 'a', 'c'])
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

/tmp/ipykernel_63493/220899500.py:2: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])


ValueError: cannot reindex on an axis with duplicate labels

In [36]:
type(test)

pandas.core.frame.DataFrame

In [35]:
x

Timestamp('2022-12-06 09:15:00.030000')

In [43]:
res = x.groupby([pd.Grouper(key='buyno',freq=100,closed='right',label='right',dropna=False)])

ValueError: Invalid frequency: 100

In [41]:
grouped_df = x.groupby([pd.Grouper(key='datetime',freq=f'M',closed='right',label='right',dropna=False),'securityid','bs'])

for key, item in grouped_df:
    print(grouped_df.get_group(key), "\n\n")

           securityid    updatetime  tradp  tradv   buyno  sellno bs  \
34109032  000001.XSHE  09:25:00.000  13.34  100.0  174034  139157  B   

             localtime      date            datetime  tradamt  
34109032  09:25:00.223  20221206 2022-12-06 09:25:00   1334.0   


           securityid    updatetime  tradp  tradv   buyno  sellno bs  \
34109031  000001.XSHE  09:25:00.000  13.34  100.0  119565  139157  S   
34109033  000001.XSHE  09:25:00.000  13.34  100.0  174034  187249  S   
34109035  000001.XSHE  09:25:00.000  13.34  100.0  174034  214919  S   
34109036  000001.XSHE  09:25:00.000  13.34  100.0  174034  221165  S   

             localtime      date            datetime  tradamt  
34109031  09:25:00.223  20221206 2022-12-06 09:25:00   1334.0  
34109033  09:25:00.223  20221206 2022-12-06 09:25:00   1334.0  
34109035  09:25:00.223  20221206 2022-12-06 09:25:00   1334.0  
34109036  09:25:00.223  20221206 2022-12-06 09:25:00   1334.0   




In [9]:
res = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].count().to_frame()
res.columns = ['cjbs']
res['bcjbs'] = aa_B.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].count()
res['scjbs'] = aa_S.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].count()
res['volume_nfq'] = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradv'].sum()
res['bvolume_nfq'] = aa_B.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradv'].sum()
res['svolume_nfq'] = aa_S.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradv'].sum()
res['amount'] = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradamt'].sum()
res['bamount'] = aa_B.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradamt'].sum()
res['samount'] = aa_S.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradamt'].sum()
res['closeprice_nfq'] = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].last()
res['openprice_nfq'] = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].first()
res['highprice_nfq'] = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].max()
res['lowprice_nfq'] = aa.groupby([pd.Grouper(key='datetime',freq=f'T',closed='right',label='right',dropna=False),'securityid'])['tradp'].min()

In [10]:
res.reset_index()

,datetime,securityid,cjbs,bcjbs,scjbs,volume_nfq,bvolume_nfq,svolume_nfq,amount,bamount,samount,closeprice_nfq,openprice_nfq,highprice_nfq,lowprice_nfq
0,2022-12-06 09:25:00,000001.XSHE,536,83,453,1157000.0,178900.0,978100.0,15434380.00,2386526.00,13047854.00,13.34,13.34,13.34,13.34
1,2022-12-06 09:31:00,000001.XSHE,2771,1353,1418,5269827.0,2297461.0,2972366.0,70157672.20,30600832.25,39556839.95,13.28,13.34,13.39,13.28
2,2022-12-06 09:32:00,000001.XSHE,1371,584,787,1918500.0,566000.0,1352500.0,25514695.00,7529410.00,17985285.00,13.31,13.28,13.33,13.27
3,2022-12-06 09:33:00,000001.XSHE,1191,504,687,1325300.0,443400.0,881900.0,17631347.00,5901141.00,11730206.00,13.31,13.30,13.34,13.28
4,2022-12-06 09:34:00,000001.XSHE,1750,565,1185,3507100.0,877800.0,2629300.0,46585686.00,11664731.00,34920955.00,13.28,13.31,13.33,13.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2022-12-06 14:54:00,000001.XSHE,308,125,183,336385.0,117085.0,219300.0,4510688.85,1570314.85,2940374.00,13.41,13.42,13.42,13.40
233,2022-12-06 14:55:00,000001.XSHE,296,174,122,618040.0,249640.0,368400.0,8290332.80,3349723.80,4940609.00,13.42,13.41,13.43,13.40
234,2022-12-06 14:56:00,000001.XSHE,404,246,158,455900.0,252940.0,202960.0,6121372.60,3397037.40,2724335.20,13.43,13.42,13.44,13.41
235,2022-12-06 14:57:00,000001.XSHE,296,143,153,389400.0,159353.0,230047.0,5226435.88,2139583.14,3086852.74,13.41,13.42,13.44,13.40
